In [ ]:
#Question 1. PySpark version (1 point)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/05 19:29:58 WARN Utils: Your hostname, ZYX resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/05 19:29:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/05 19:29:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/05 19:29:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.version

'3.3.2'

In [ ]:
#Question 2. Size of partition (1 point)

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-05 19:32:15--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
3.161.178.218, 3.161.178.162, 3.161.178.24, ...vzurychx.cloudfront.net)... 
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.161.178.218|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  24.2MB/s    in 2.5s    

2025-03-05 19:32:18 (24.2 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [9]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [12]:
# Read the Parquet file
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

# Repartition the DataFrame into 24 partitions
df = df.repartition(24)

# Write the DataFrame to Parquet with compression
df.write.parquet('data/pq/fhvhv/2021/02/', mode='overwrite', compression='snappy')

[Stage 4:===================================================>       (7 + 1) / 8]

25/03/05 19:39:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


[Stage 6:===================>                                      (8 + 8) / 24]

25/03/05 19:39:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


[Stage 6:======================================>                  (16 + 8) / 24]

25/03/05 19:39:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [13]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

df = spark.read.parquet('yellow_tripdata_2024-10.parquet')
df = df.repartition(4)
df.write.parquet('homework/2')

In [ ]:
#Question 3. Number of trips (1 point)

In [15]:
df.registerTempTable('trips_data')  

spark.sql("""

SELECT count(1) FROM trips_data 
WHERE DATE(tpep_pickup_datetime) = '2024-10-15';
""").show()

+--------+
|count(1)|
+--------+
|  128909|
+--------+



In [ ]:
#Question 5: What is the length of the longest trip in the dataset in hours?

In [16]:
spark.sql("""
SELECT 
    (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS trip_duration
FROM trips_data
ORDER BY trip_duration DESC
LIMIT 1;
""").show()                  

[Stage 23:====================================>                     (5 + 3) / 8]

+------------------+
|     trip_duration|
+------------------+
|162.61777777777777|
+------------------+



In [ ]:
#Question 6: Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [18]:
lookup_df = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')

lookup_df.registerTempTable('lookup')  

spark.sql("""

SELECT lookup.Zone , count(1) as num_trips FROM trips_data 
INNER JOIN lookup ON lookup.LocationID = trips_data.PULocationID
GROUP BY lookup.Zone
ORDER BY num_trips ASC;
""").show()  

[Stage 28:====================================>                     (5 + 3) / 8]

+--------------------+---------+
|                Zone|num_trips|
+--------------------+---------+
|Governor's Island...|        1|
|       Arden Heights|        2|
|       Rikers Island|        2|
|         Jamaica Bay|        3|
| Green-Wood Cemetery|        3|
|   Rossville/Woodrow|        4|
|Charleston/Totten...|        4|
|       West Brighton|        4|
|Eltingville/Annad...|        4|
|       Port Richmond|        4|
|        Crotona Park|        6|
|         Great Kills|        6|
|     Mariners Harbor|        7|
|Heartland Village...|        7|
|Saint George/New ...|        9|
|             Oakwood|        9|
|       Broad Channel|       10|
|New Dorp/Midland ...|       10|
|         Westerleigh|       12|
|     Pelham Bay Park|       12|
+--------------------+---------+
only showing top 20 rows

